In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd
#from HEB_scraping_filter import *


In [2]:
#####################################################
################ utility function
def return_string(string, start_after_word,stop_before_word):
    """return part of string between 2 words     (string, start_after_word,stop_befor_word)"""
    try:
        if start_after_word=="":
            start_address=0
        else:
            start_address=re.search(start_after_word,string).span()[1]
        #print(start_address)
        if stop_before_word=="":
            stop_address=len(string)
        else:
            stop_address=re.search(stop_before_word,string).span()[0]
        #print(stop_address)
        my_string=string[start_address:stop_address]
    except Exception as e:
        # handle error
        error=f"STRING _exception_: {type(e).__name__},</br> _arguments_: {e.args}"
        #print(error)
        my_string=None
    return my_string


# In[7]:


####################################################
###############filter function 
def heb_filter(soup_find):
    """ function takes result of HEB scraping - soup.find_all and
    transforms it to dataframe """
    my_result=soup_find
    scrap_df=pd.DataFrame()
#   my_result=soup.body.find_all('a',id=re.compile('product-'))
       
    i=0
    for record in my_result:
        json=eval(record.find('script',type="application/ld+json").string)
        try:
            try:
                name=record.find('span',class_=re.compile('responsive')).text.strip()
            except:
                name=json['name']
            size=return_string(name,"Milk","")
            json=eval(record.find('script',type="application/ld+json").string)
            id=json['id']
            brand=json['brand']
            category=json['category']
            price=json['price']
            search_string=record['aria-label'].strip()
            features=return_string(search_string,"Features:","")
         #  rating=return_string(search_string,"Rated","stars")
            try:
                uomSalePrice=record.find('span',class_=re.compile('uomSalePrice')).text.strip()
            except:
                uomSalePrice=None
            try:
                image=record.find('img',attrs={"data-src":re.compile('prd-small')})['data-src']
            except:
                image=record.find('img',attrs={"src":re.compile('prd-small')})['src']
            if record.find('img',src=re.compile('coupon'))==None:
                coupon=0
            else:
                coupon=1
            #debug prints
            #print(i) 
            #print (f'{name} \n{size}\n{json}\n{id}\n{brand}\n{category}\n{price}\n{features}\n{coupon}\n{uomSalePrice}\n{image}')
            #print('___________________')
            i+=1
            scrap_df = scrap_df.append({'id':id,
                                'name': name,
                                'brand':brand,
                                'size':size,
                                'category':category,
                                'price':price,
                                'features':features,
                                'coupon':coupon,
                                'uomSalePrice':uomSalePrice,
                                'image':image,
                               }, ignore_index=True)
        except Exception as e:
        # handle error
            print(i) 
            print (f'{name} \n{size}\n{json}\n{id}\n{brand}\n{category}\n{price}\n{features}\n{coupon}\n{uomSalePrice}\n{image}')
            error=f"MAIN_exception_: {type(e).__name__},</br> _arguments_: {e.args}"
            print(error)
        
    return scrap_df

In [3]:
#setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/Baud/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [4]:
#loading the first page of a search for milk in the web browser
milk = "https://www.heb.com/search/results?N=3139289889&No=0&Nrpp=60&Ntt=milk&q=milk&prodFilter=none"
browser.visit(milk)

In [5]:
#scraping through the first page of results
html = browser.html
soup = bs(html,"lxml")

page = soup.body.find_all('a',id=re.compile('product-'))
milks = page
#had to use find by xpath because one result had the word "Next" in the text
browser.find_by_xpath('//a[@aria-label="go to next page"]').click()
time.sleep(1)
#using a test for whether the next button is present to continue through the rest of the results
while browser.is_element_present_by_xpath('//a[@aria-label="go to next page"]'):
    html = browser.html
    soup = bs(html,"lxml")
    page = soup.body.find_all('a',id=re.compile('product-'))
    milks.extend(page)
    try:
        browser.find_by_xpath('//a[@aria-label="go to next page"]').click()
        time.sleep(1)
    except:
        break

In [6]:
#quitting the browser to save RAM
browser.quit()

In [8]:
record=milks[660]
record.find('img',attrs={"src":re.compile('prd-small')})['src']

'https://images.heb.com/is/image/HEBGrocery/prd-small/similac-go-grow-non-gmo-toddler-drink-powder-002070468.jpg'

In [126]:
json=eval(record.find('script',type="application/ld+json").string)
#name=record.find('span',class_=re.compile('responsive')).text.strip()
name=json['name']
size=return_string(name,"Milk","")

id=json['id']
brand=json['brand']
category=json['category']
price=json['price']
search_string=record['aria-label'].strip()
features=return_string(search_string,"Features:","")
# #  rating=return_string(search_string,"Rated","stars")
uomSalePrice=record.find('span',class_=re.compile('uomSalePrice')).text.strip()
#image=record.find('img',attrs={"data-src":re.compile('prd-small')})['data-src']
print (f'{name}\n{size}\n{json}\n{id}\n{brand}\n{category}\n{price}')
#\n{id}\n{brand}\n{category}\n{price}\n{features}\n{coupon}\n{uomSalePrice}\n{image}')

Similac Go & Grow Non-GMO Toddler Drink Powder
None
{'name': 'Similac Go & Grow Non-GMO Toddler Drink Powder', 'id': '2070468', 'category': 'Baby & Kids/Food & Formula/Formula', 'brand': 'Similac', 'variant': 'each', 'price': 20.58, 'list': 'We recommend', 'position': '1', 'dimension26': 'false', 'dimension40': 'default'}
2070468
Similac
Baby & Kids/Food & Formula/Formula
20.58


In [9]:
scrap_df=heb_filter(milks)

In [10]:
heb_df=scrap_df.loc[scrap_df['category']=='Dairy & Eggs/Milk']

In [11]:
heb_df.loc[heb_df['name']=='Thai Kitchen Organic Unsweetened Coconut Milk, 13.66 oz']

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
97,Thai Kitchen,Dairy & Eggs/Milk,0.0,LESS THAN 5G NET CARBS. VEGETARIAN. VEGAN....,740626,https://images.heb.com/is/image/HEBGrocery/prd...,"Thai Kitchen Organic Unsweetened Coconut Milk,...",2.46,", 13.66 oz",($0.19/oz)


In [12]:
path='data/heb_scrape.csv'
scrap_df.to_csv(path, index_label=False)

In [13]:
scrap_df.head(50)

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,", 1 gal",($0.05/oz)
1,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,"fat Milk, 1 gal",($0.05/oz)
2,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS. Primo Pick.,671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,", 1/2 gal",($0.07/oz)
3,H-E-B,Dairy & Eggs/Milk,1.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.88,", 1/2 gal",($0.03/oz)
4,H-E-B,Dairy & Eggs/Milk,1.0,GO LOCAL. SELECT INGREDIENTS.,314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.88,", 1/2 gal",($0.03/oz)
5,H-E-B,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,671075,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,", 1/2 gal",($0.07/oz)
6,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. Primo Pick.,730555,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B MooTopia Lactose Free Chocolate 2% Reduc...,4.10,", 1/2 gal",($0.07/oz)
7,Central Market,Dairy & Eggs/Milk,0.0,KOSHER. LOW SODIUM. VEGETARIAN. ORGANIC. ...,1770593,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics 2% Reduced Fat Milk, 1 qt",2.04,", 1 qt",($0.07/oz)
8,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314129,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Low Fat 1% Milk, 1 gal",3.38,", 1 gal",($0.03/oz)
9,H-E-B,Dairy & Eggs/Milk,1.0,GO LOCAL. SELECT INGREDIENTS.,481244,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Reduced Fat 2% Milk, ...",1.88,", 1/2 gal",($0.03/oz)
